In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import collections
import helper
import numpy as np
import io
import pandas as pd
import string
import re

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd

print( tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:
data_path='sample_data/data_training'

In [ ]:
import os
data_paths = [os.path.join(pth, f)
for pth, dirs, files in os.walk(data_path) for f in files]
data_paths

['sample_data/data_training/corpus_kelas.txt']

In [ ]:
data_path1='sample_data/data_training/corpus_kelas.txt'

In [ ]:
input_texts = []
target_texts = []
with open(data_path1) as f:
    lines = f.read().split('\n')
for line in lines[: min(250, len(lines) - 1)]:
    input_text = line.split('\t')[0]
    target_text = line.split('\t')[1]
    input_texts.append(input_text)
    target_texts.append(target_text)

In [ ]:
print('type of input_text',type(input_text))
print('type of target_texts',type(target_texts))

type of input_text <class 'str'>
type of target_texts <class 'list'>


In [ ]:
##converting the list in pandas dataframe since input_text,target_text are both are type of list
zippedList =  list(zip(input_texts, target_texts))
lines = pd.DataFrame(zippedList, columns = ['input' , 'output'])
lines.head()

,input,output
0,When do you eat Soto Lamongan?,Wes kangen banget karo rasane Soto Lamongan si...
1,How does Soto Lamongan taste?,Aku seneng banget karo panganan Soto Lamongan ...
2,where will he eat pempek?,"dheweke milih ngrasakake khas daerah, yaiku pe..."
3,where do they eat chicken?,padha seneng banget karo masakan pitik ing War...
4,where does he drink dawet?,dheweke sregep ngrasakake segelas dawet ijo se...


In [ ]:
input_lines = list()
for line in lines.input:
    input_lines.append( line )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( input_lines )
tokenized_input_lines = tokenizer.texts_to_sequences( input_lines )

length_list = list()
for token_seq in tokenized_input_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Input max length is {}'.format( max_input_length ))

padded_input_lines = preprocessing.sequence.pad_sequences( tokenized_input_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_input_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

input_word_dict = tokenizer.word_index
num_input_tokens = len( input_word_dict )+1
print( 'Number of Input tokens = {}'.format( num_input_tokens))

Input max length is 26
Encoder input data shape -> (250, 26)
Number of Input tokens = 511


In [ ]:
encoder_input_data

array([[ 42,   5,   3, ...,   0,   0,   0],
       [  9,  10,  93, ...,   0,   0,   0],
       [ 23, 163, 164, ...,   0,   0,   0],
       ...,
       [  1,  18,   4, ...,   0,   0,   0],
       [  1,   6,   2, ...,   0,   0,   0],
       [  5,   3,  39, ...,   0,   0,   0]], dtype=int32)

In [ ]:
output_lines = list()
for line in lines.output:
    output_lines.append( '<START> ' + line + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( output_lines )
tokenized_output_lines = tokenizer.texts_to_sequences( output_lines )

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines = preprocessing.sequence.pad_sequences( tokenized_output_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_output_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict = tokenizer.word_index
num_output_tokens = len( output_word_dict )+1
print( 'Number of Output tokens = {}'.format( num_output_tokens))


Output max length is 39
Decoder input data shape -> (250, 39)
Number of Output tokens = 833


In [ ]:
decoder_target_data = list()
for token_seq in tokenized_output_lines:
    decoder_target_data.append( token_seq[ 1 : ] )

padded_output_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_output_lines = utils.to_categorical( padded_output_lines , num_output_tokens )
decoder_target_data = np.array(onehot_output_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (250, 39, 833)


In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax )
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            130816    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            213248    ['input_2[0][0]']             
                                                                                              

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=124, epochs=500)
model.save( 'model.h5' )

Epoch 1/500
3/3 [==============================] - 15s 472ms/step - loss: 6.7231
Epoch 2/500
3/3 [==============================] - 1s 457ms/step - loss: 6.7063
Epoch 3/500
3/3 [==============================] - 1s 399ms/step - loss: 6.6613
Epoch 4/500
3/3 [==============================] - 1s 424ms/step - loss: 6.3710
Epoch 5/500
3/3 [==============================] - 1s 442ms/step - loss: 5.8130
Epoch 6/500
3/3 [==============================] - 1s 396ms/step - loss: 5.6404
Epoch 7/500
3/3 [==============================] - 2s 767ms/step - loss: 5.5239
Epoch 8/500
3/3 [==============================] - 2s 689ms/step - loss: 5.4523
Epoch 9/500
3/3 [==============================] - 1s 370ms/step - loss: 5.4366
Epoch 10/500
3/3 [==============================] - 2s 666ms/step - loss: 5.4225
Epoch 11/500
3/3 [==============================] - 1s 502ms/step - loss: 5.4024
Epoch 12/500
3/3 [==============================] - 1s 490ms/step - loss: 5.3773
Epoch 13/500
3/3 [==================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def make_inference_models():

    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model , decoder_model

In [ ]:
import tensorflow as tf
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] )
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()
for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word

        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]

        reversed_translation = ' '.join(decoded_translation.split()[::-1])

    print( "Bot:" +reversed_translation.replace('ini kebalikan > ', '') )
    print()

User: How does Klappertaart taste
1/1 [==============================] - 0s 23ms/step
Bot:end seleraku karo cocok banget enak rasane ndadekake sing lembut sing tekstur lan khas sing klapa rasa nduweni klappertaart

User: How does Es Doger taste
1/1 [==============================] - 0s 38ms/step
Bot:end ketan bal saka kenyal tekstur seneng aku seger lan legi rasane banget seger sing minuman minangka doger es

